Will follow [Huggingface's translation tutorial](https://huggingface.co/docs/transformers/tasks/translation) more or less.

In [ ]:
# Stuff for running the same notebook locally and on Google Colab for training
import sys
IN_COLAB = 'google.colab' in sys.modules
fp = "/content/drive/MyDrive/Colab Notebooks/ersetzBERT/training_data_gender.jsonl" if IN_COLAB else "../data/training_data_gender.jsonl"
if IN_COLAB:
    from google.colab import drive # type: ignore
    drive.mount('/content/drive')

In [ ]:
%pip install transformers datasets sacrebleu

In [ ]:
from datasets import load_dataset

data = load_dataset("json", data_files=fp)
data = data["train"].train_test_split(test_size=0.2, shuffle=False)
data["train"][:3]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")

In [ ]:
def preprocess_function(data):
    inputs = [f"Ersetze {a} durch {b}: {x}" for a, b, x in zip(data["a"], data["b"], data["x"])]
    targets = data["y"]
    model_inputs = tokenizer(inputs, max_length=256, truncation=True)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=256, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_data = data.map(preprocess_function, batched=True)

In [ ]:
from transformers import TFAutoModelForSeq2SeqLM

model = TFAutoModelForSeq2SeqLM.from_pretrained("t5-small")

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, return_tensors="tf")

In [ ]:
tf_train_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_test_set = tokenized_data["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
from transformers import create_optimizer, AdamWeightDecay

optimizer = AdamWeightDecay(learning_rate=2e-5, weight_decay_rate=0.01)

In [ ]:
model.compile(optimizer=optimizer)

In [ ]:
if IN_COLAB:
    model.fit(x=tf_train_set, validation_data=tf_test_set, epochs=3)
    model.save("/content/drive/MyDrive/Colab Notebooks/ersetzBERT/model")

In [ ]:
from transformers import pipeline

generator = pipeline(task="text2text-generation", model=model, tokenizer=tokenizer)

In [ ]:
entries = list(zip(data["test"]["x"], data["test"]["a"], data["test"]["b"], data["test"]["y"]))

In [ ]:
for x, a, b, y in entries[:5]:
    prompt = f"Ersetze {a} durch {b}: {x}"
    print(prompt)
    prediction = generator(prompt)[0]["generated_text"]
    print(prediction)
    print(len(prompt), len(prediction), len(prompt) + len(prediction))
    print()